# Using the IBM FHIR Server Location Search in Jupyter

**Author**: [Paul Bastide](mailto:pbastide@us.ibm.com)

The IBM FHIR Server supports FHIR Search with the SearchParameter `near` on the Location API. 

This notebook shows how to use the IBM FHIR Server examples with Location Search and use folium to render their plots.

More advanced demonstrations can be generated from this location data which reference these Locations, and are retrieved using [FHIR Search - Reference](https://www.hl7.org/fhir/search.html#reference).

In [6]:
# This snippet install geojson and geopandas to faciliate rendering in folium. 

# geojson to process the GIS data
!pip3 install geojson

#http://geopandas.org/
!pip3 install git+git://github.com/geopandas/geopandas.git
    
!pip3 install folium

  Cloning git://github.com/geopandas/geopandas.git to /tmp/wsuser/pip-req-build-t95wez_c
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/wsuser/pip-req-build-t95wez_c
  Created wheel for geopandas: filename=geopandas-0.8.0+63.gcdb4282-py2.py3-none-any.whl size=971092 sha256=78a353d56c98e2f560caa2b65d03ab61cdea4de3f5742f2b010c63deb4b8ebc0
  Stored in directory: /tmp/wsuser/pip-ephem-wheel-cache-wabwd2pm/wheels/cf/3e/0b/6475054094c2b1ea054158ac1fdcf749fb92f5b512377e4cf8
Successfully built geopandas
     |████████████████████████████████| 93 kB 3.8 MB/s  eta 0:00:01


In [14]:
# The following imports are used in the analysis of the location data:

import os

# imported to retrieve any secondary data
import requests

# import pandas to process the data (and json_normalize to use the geojson as a plain dataframe)
import pandas as pd
import numpy as np

# Import to control the output of the choropleth map
import matplotlib.cm as cm
import matplotlib.colors as colors

# standard matplot lib 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates

# Import Folium to visualize the map
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

# Import the GeoPandas utility to process / use establish the polygon area
import geopandas 

# figures out the neighborhoods for each coordinate. 
from shapely.geometry import Point 
#(switched to geojson point to more precisely identify location)
#from geojson import Point

# Imports the Clustering Utilities
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import json
from pandas.io.json import json_normalize

# used to process fhir-examples
import zipfile, io
import urllib3

print("All imports complete")

# a function to print the top x rows and add a newline
def peek(string, line_count=25):
    print(os.linesep.join(string.split(os.linesep)[:line_count]) + '\n')

All imports complete


# Query the IBM FHIR Server with near

In [42]:
base = 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open'

# Get the Location using Requests
headers = {
    'Content-Type': 'application/json'
}

# Search within 10Km of Cambridge Massachusetts
queryParams = {
    'near': '42.373611|-71.110558|10|km',
    "_count" : 200
}

# Search Parameter = near=
resp_loc = requests.get(base + '/Location', headers=headers, params=queryParams)
peek(resp_loc.text, 68)

{
    "resourceType": "Bundle",
    "id": "799c7d13-2eab-4211-b110-051ede0ea3ec",
    "type": "searchset",
    "total": 22,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Location?_count=200&near=42.373611%7C-71.110558%7C10%7Ckm&_page=1"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Location/175db647821-ac68d6f1-7d4c-4be7-b176-9d2d033a8a03",
            "resource": {
                "resourceType": "Location",
                "id": "175db647821-ac68d6f1-7d4c-4be7-b176-9d2d033a8a03",
                "meta": {
                    "versionId": "1",
                    "lastUpdated": "2020-11-18T12:46:36.065Z"
                },
                "identifier": [
                    {
                        "id": "9bb2a06a-358c-4219-8a58-cfc81

In [49]:
# Convert to a list
response_loc_json = json.loads(resp_loc.text)

location_rows = []
for location in response_loc_json['entry']:
    row = pd.json_normalize(location)
    location_rows.append(row)
    
# If you are debugging... print(location_rows)

In [50]:
# zoom 15 is used to show the groupings
cambridge = [ 42.373611, -71.11000]
map_cambridge_locs_from_server = folium.Map(location=cambridge, zoom_start=10)

# Iterate through the Rows
for location_row in location_rows :
        # print(location_row)
        
        # Cast the values into the appropriate types as FOLIUM will die weirdly without it. 
        lat_inc = float(location_row['resource.position.latitude'])
        long_inc = float(location_row['resource.position.longitude'])
        name_inc = str(location_row['resource.name'])
        
        label = folium.Popup(name_inc, parse_html=True)
        folium.CircleMarker(
            [lat_inc, long_inc],
            radius=5,
            popup=label,
            fill=True,
            fill_color='red',
            fill_opacity=0.7).add_to(map_cambridge_locs_from_server)
        
map_cambridge_locs_from_server

(C) Copyright IBM Corp. 2020

SPDX-License-Identifier: Apache-2.0